### Imports and Options

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import os

In [22]:
from dotenv import load_dotenv 

In [23]:
load_dotenv()

True

In [25]:
from census import Census

### Testing Census Class

In [43]:
census = Census()

In [44]:
results = census.download()

In [45]:
results["acs/acs5/profile"]

,Year,NAME,Total households estimate,Total households MOE,zip code tabulation area
0,2022,ZCTA5 60620,26150,816,60620
1,2022,ZCTA5 60643,18197,776,60643
2,2022,ZCTA5 60652,13643,666,60652


In [46]:
results["acs/acs5"]

,Year,NAME,Pop:MOE,Pop:MOE,,MOE,Owner occupied:,Owner occupied:MOE,Renter occupied:,Renter occupied:MOE,...,Renter-occupied housing units:!!10 to 19:MOE,Renter-occupied housing units:!!20 to 49,Renter-occupied housing units:!!20 to 49:MOE,Renter-occupied housing units:!!50 or more,Renter-occupied housing units:!!50 or more:MOE,Renter-occupied housing units:!!Mobile home,zip code tabulation area,Renter-occupied housing units:!!Mobile home:MOE,"Renter-occupied housing units:!!Boat, RV, van, etc.:","Renter-occupied housing units:!!Boat, RV, van, etc.:MOE"
0,2022,ZCTA5 60620,68330,68330,26150,816,12697,632,13453,702,...,291,857,311,1028,234,30,60620,46,0,29
1,2022,ZCTA5 60643,48270,48270,18197,776,13531,763,4666,599,...,147,196,114,360,137,31,60643,50,0,26
2,2022,ZCTA5 60652,42223,42223,13643,666,11585,677,2058,397,...,85,0,26,102,83,0,60652,26,0,26


### Connecting to the Census

In [4]:
# Read in api keys
# with open('api_keys.txt', 'r') as file:
#     file_keys = file.readlines()
# file_keys
# key = str(file_keys[0]).strip()

key = os.environ["CENSUS_API_KEY"]

In [5]:
# Chicago Zip Codes
zip_codes = ['60620', '60652', '60643']

In [6]:
################################################################################
# Define Variables Used In API Request
################################################################################

# Set variables for API endpoint and key
endpoint = "https://api.census.gov/data"

# The API limits the number of variables to 50 at a time
MAX_CODE_CHUNK = 49

years = [2022]

variables = [
    # ACS Profile Data
    {
        'dataset':'acs/acs5/profile',
        'time':years,
        'codes':{
            'Total households estimate':'DP02_0001E',
            'Total households MOE':'DP02_0001M'
        }
    },
    # ACS Detail Data
    {
        'dataset':'acs/acs5',
        'time':years,
        'codes':{
            'Estimate!!Total:Pop':'B01001_001E',
            'Estimate!!Total:Pop:MOE':'B01001_001E',
            'Estimate!!Total:':'B25003_001E',
            'Estimate!!Total:MOE':'B25003_001M',
            'Estimate!!Total:!!Owner occupied:':'B25003_002E',
            'Estimate!!Total:!!Owner occupied:MOE':'B25003_002M',
            'Estimate!!Total:!!Renter occupied:':'B25003_003E',
            'Estimate!!Total:!!Renter occupied:MOE':'B25003_003M',
            'Estimate!!Total:':'B25032_001E',
            'Estimate!!Total:MOE':'B25032_001M',
            'Estimate!!Total:!!Owner-occupied housing units:':'B25032_002E',
            'Estimate!!Total:!!Owner-occupied housing units:MOE':'B25032_002M',
            'Estimate!!Total:!!Owner-occupied housing units:!!1, detached:':'B25032_003E',
            'Estimate!!Total:!!Owner-occupied housing units:!!1, detached:MOE':'B25032_003M',
            'Estimate!!Total:!!Owner-occupied housing units:!!1, attached, detached':'B25032_004E',
            'Estimate!!Total:!!Owner-occupied housing units:!!1, attached:MOE':'B25032_004M',
            'Estimate!!Total:!!Owner-occupied housing units:!!2, detached':'B25032_005E',
            'Estimate!!Total:!!Owner-occupied housing units:!!2:MOE':'B25032_005M',
            'Estimate!!Total:!!Owner-occupied housing units:!!3 or 4:':'B25032_006E',
            'Estimate!!Total:!!Owner-occupied housing units:!!3 or 4:MOE':'B25032_006M',
            'Estimate!!Total:!!Owner-occupied housing units:!!5 to 9:':'B25032_007E',
            'Estimate!!Total:!!Owner-occupied housing units:!!5 to 9:MOE':'B25032_007M',
            'Estimate!!Total:!!Owner-occupied housing units:!!10 to 19:':'B25032_008E',
            'Estimate!!Total:!!Owner-occupied housing units:!!10 to 19:MOE':'B25032_008M',
            'Estimate!!Total:!!Owner-occupied housing units:!!20 to 49:':'B25032_009E',
            'Estimate!!Total:!!Owner-occupied housing units:!!20 to 49:MOE':'B25032_009M',
            'Estimate!!Total:!!Owner-occupied housing units:!!50 or more:':'B25032_010E',
            'Estimate!!Total:!!Owner-occupied housing units:!!50 or more:MOE':'B25032_010M',
            'Estimate!!Total:!!Owner-occupied housing units:!!Mobile home:':'B25032_011E',
            'Estimate!!Total:!!Owner-occupied housing units:!!Mobile home:MOE':'B25032_011M',
            'Estimate!!Total:!!Owner-occupied housing units:!!Boat, RV, van, etc.:':'B25032_012E',
            'Estimate!!Total:!!Owner-occupied housing units:!!Boat, RV, van, etc.:MOE':'B25032_012M',
            'Estimate!!Total:!!Renter-occupied housing units:':'B25032_013E',
            'Estimate!!Total:!!Renter-occupied housing units:MOE':'B25032_013M',
            'Estimate!!Total:!!Renter-occupied housing units:!!1, detached':'B25032_014E',
            'Estimate!!Total:!!Renter-occupied housing units:!!1, detached:MOE':'B25032_014M',
            'Estimate!!Total:!!Renter-occupied housing units:!!1, attached':'B25032_015E',
            'Estimate!!Total:!!Renter-occupied housing units:!!1, attached:MOE':'B25032_015M',
            'Estimate!!Total:!!Renter-occupied housing units:!!2':'B25032_016E',
            'Estimate!!Total:!!Renter-occupied housing units:!!2:MOE':'B25032_016M',
            'Estimate!!Total:!!Renter-occupied housing units:!!3 or 4':'B25032_017E',
            'Estimate!!Total:!!Renter-occupied housing units:!!3 or 4:MOE':'B25032_017M',
            'Estimate!!Total:!!Renter-occupied housing units:!!5 to 9':'B25032_018E',
            'Estimate!!Total:!!Renter-occupied housing units:!!5 to 9:MOE':'B25032_018M',
            'Estimate!!Total:!!Renter-occupied housing units:!!10 to 19':'B25032_019E',
            'Estimate!!Total:!!Renter-occupied housing units:!!10 to 19:MOE':'B25032_019M',
            'Estimate!!Total:!!Renter-occupied housing units:!!20 to 49':'B25032_020E',
            'Estimate!!Total:!!Renter-occupied housing units:!!20 to 49:MOE':'B25032_020M',
            'Estimate!!Total:!!Renter-occupied housing units:!!50 or more':'B25032_021E',
            'Estimate!!Total:!!Renter-occupied housing units:!!50 or more:MOE':'B25032_021M',
            'Estimate!!Total:!!Renter-occupied housing units:!!Mobile home':'B25032_022E',
            'Estimate!!Total:!!Renter-occupied housing units:!!Mobile home:MOE':'B25032_022M',
            'Estimate!!Total:!!Renter-occupied housing units:!!Boat, RV, van, etc.:':'B25032_023E',
            'Estimate!!Total:!!Renter-occupied housing units:!!Boat, RV, van, etc.:MOE':'B25032_023M',
        }
    }
]

In [10]:
# Load necessary libraries
import requests
import time
import pandas as pd
from urllib.parse import quote


################################################################################
# Loop Through Variables & Make API Call & Collect Results
################################################################################
for vars in variables:
    time_results = []
    for year in vars['time']:
        chunk_results = pd.DataFrame(columns=['NAME'])
        for i in range(0, len(vars['codes'].values()), MAX_CODE_CHUNK):
            codes = list(vars['codes'].values())
            codes_subset = ['NAME'] + codes[i:i + MAX_CODE_CHUNK]
        
            # Construct API request URL
            location = "zip code tabulation area:" + ','.join(zip_codes)
            url_vars = ','.join(codes_subset)
            dataset = vars['dataset']
            url = f"{endpoint}/{year}/{dataset}?get={url_vars}&for={location}&key={key}"
            print(url)
            encoded_url = quote(url, safe=':/?=&')
            
            # Send API request and convert response to data frame
#             print(encoded_url)
            response = requests.get(encoded_url)
            df_response = pd.DataFrame(response.json()[1:], columns=response.json()[0])
            chunk_results = pd.merge(chunk_results, df_response, how='outer')
            
            # Sleep for 1/4 sec to avoid rate limits
            time.sleep(0.25)
        
        # Add the results of each year to list
        chunk_results.insert(0, 'Year', year)
        time_results.append(chunk_results)

    # Combine all of the data into one Pandas Table
    final_results = pd.concat(time_results, axis=0)


https://api.census.gov/data/2022/acs/acs5/profile?get=NAME,DP02_0001E,DP02_0001M&for=zip code tabulation area:60620,60652,60643&key=e4ba254c7ccb616296fa1ecf9f0cf07d4df55ab2
https://api.census.gov/data/2022/acs/acs5?get=NAME,B01001_001E,B01001_001E,B25032_001E,B25032_001M,B25003_002E,B25003_002M,B25003_003E,B25003_003M,B25032_002E,B25032_002M,B25032_003E,B25032_003M,B25032_004E,B25032_004M,B25032_005E,B25032_005M,B25032_006E,B25032_006M,B25032_007E,B25032_007M,B25032_008E,B25032_008M,B25032_009E,B25032_009M,B25032_010E,B25032_010M,B25032_011E,B25032_011M,B25032_012E,B25032_012M,B25032_013E,B25032_013M,B25032_014E,B25032_014M,B25032_015E,B25032_015M,B25032_016E,B25032_016M,B25032_017E,B25032_017M,B25032_018E,B25032_018M,B25032_019E,B25032_019M,B25032_020E,B25032_020M,B25032_021E,B25032_021M,B25032_022E&for=zip code tabulation area:60620,60652,60643&key=e4ba254c7ccb616296fa1ecf9f0cf07d4df55ab2
https://api.census.gov/data/2022/acs/acs5?get=NAME,B25032_022M,B25032_023E,B25032_023M&for=zip c

In [8]:
final_results.to_csv('census_dwnld.csv', index=False)

In [9]:
final_results

,Year,NAME,B01001_001E,B01001_001E,B25032_001E,B25032_001M,B25003_002E,B25003_002M,B25003_003E,B25003_003M,...,B25032_019M,B25032_020E,B25032_020M,B25032_021E,B25032_021M,B25032_022E,zip code tabulation area,B25032_022M,B25032_023E,B25032_023M
0,2022,ZCTA5 60620,68330,68330,26150,816,12697,632,13453,702,...,291,857,311,1028,234,30,60620,46,0,29
1,2022,ZCTA5 60643,48270,48270,18197,776,13531,763,4666,599,...,147,196,114,360,137,31,60643,50,0,26
2,2022,ZCTA5 60652,42223,42223,13643,666,11585,677,2058,397,...,85,0,26,102,83,0,60652,26,0,26


In [11]:
test = {"one": 1, "two": 2}
len(test)

2